---
# Transformer XL
---

---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../deep-learning-dna")

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import math
import string

import BigBird
import Set_Transformer 
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utils as tfu

In [5]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [6]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/deep-learning-dna/dnabert-pretrain-ablation-dim:8dim").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model

In [7]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/dnasamples:v1").download('/data/dna_samples:v1')
samples = find_dbs(dataset_path + '/train')
samples[13]

wandb: Downloading large artifact dnasamples:v1, 4086.55MB. 1260 files... Done. 0:0:0.3


'/data/dna_samples:v1/train/WS-CCW-Jul2015_S82_L001_R1_001.db'

---
# Create Dataset

In [8]:
split_ratios = [0.8, 0.2]
set_len = 400
sequence_len = 150
kmer = 3
batch_size = 20
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [9]:
rng.shuffle(random_samples)

In [10]:
trimmed_samples, (train_dataset, val_dataset) = DnaSampleGenerator.split(samples=random_samples[0:5], split_ratios=split_ratios, subsample_length=set_len, sequence_length=sequence_len,kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

In [11]:
random_samples[0:5]

['/data/dna_samples:v1/train/WS-CCE-Apr2016_S6_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes52-10-TC_S53_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-WH-Jul2016_S46_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes41-10-HN_S42_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley026-Ag-072820_S165_L001_R1_001.db']

--- 
# Batch Parameters

In [12]:
block_size = 200
seq_len = set_len
maxlen = set_len
vocab_size = 5
num_chars_data = set_len*sequence_len
max_files = len(train_dataset.samples)
max_files

5

In [13]:
if block_size-2 > seq_len:
    raise ValueError("Block size should not be bigger than sequence length")

In [14]:
print(maxlen)
print(vocab_size)
print(num_chars_data)

400
5
60000


---
# Create Embeddings

In [15]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [16]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)

        return result
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, 128)
        return tf.reshape(encoded, (batch_size, subsample_size, -1))
    
    def call(self, data):
        return  self.modify_data_for_input(data)

---
# Setup

In [17]:
def clone_initializer(initializer):
    if isinstance(initializer, tf.keras.initializers.Initializer):
        return initializer.__class__.from_config(initializer.get_config())
    return initializer

In [18]:
mask = tf.ones(tf.shape(train_dataset[0][0])[:-1])

---
# Create Big Bird Masks

In [19]:
class Create_BigBird_Masks(keras.layers.Layer):
    def __init__(self, attention_block_size):
        super(Create_BigBird_Masks, self).__init__()
            
        self.mask_layer = BigBird.BigBirdMasks(block_size=attention_block_size)
        
    def call(self, one_batch):

        mask = tf.ones(tf.shape(one_batch)[:-1])
                       
        masks = self.mask_layer(one_batch, mask)      
        
        return masks

---
# Cache Memory

In [20]:
def _cache_memory(current_state, previous_state, memory_length, reuse_length=0):
    if memory_length is None or memory_length == 0:
        return None
    else:
        if reuse_length > 0:
            current_state = current_state[:, :reuse_length, :]

        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# XL Block

In [21]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 vocab_size,
                 hidden_size,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 norm_epsilon=1e-12,
                 inner_activation="relu",
                 kernel_initializer="variance_scaling",
                 inner_dropout=0.0,
                 **kwargs):

        super(TransformerXLBlock, self).__init__(**kwargs)
        self._vocab_size = vocab_size
        self._num_heads = num_attention_heads
        self._head_size = head_size
        self._hidden_size = hidden_size
        self._inner_size = inner_size
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._inner_activation = inner_activation
        self._norm_epsilon = norm_epsilon
        self._kernel_initializer = kernel_initializer
        self._inner_dropout = inner_dropout
        self._attention_layer_type = BigBird.BigBirdAttention
        
    def build(self, input_shape):
        input_tensor = input_shape[0] if len(input_shape) == 2 else input_shape
        input_tensor_shape = tf.TensorShape(input_tensor)
        if len(input_tensor_shape.as_list()) != 3:
            raise ValueError("TransformerLayer expects a three-dimensional input of "
                                             "shape [batch, sequence, width].")
        batch_size, sequence_length, hidden_size = input_tensor_shape

        if len(input_shape) == 2:
            mask_tensor_shape = tf.TensorShape(input_shape[1])
            expected_mask_tensor_shape = tf.TensorShape(
                    [batch_size, sequence_length, sequence_length])
            if not expected_mask_tensor_shape.is_compatible_with(mask_tensor_shape):
                raise ValueError("When passing a mask tensor to TransformerXLBlock, "
                                                 "the mask tensor must be of shape [batch, "
                                                 "sequence_length, sequence_length] (here %s). Got a "
                                                 "mask tensor of shape %s." %
                                                 (expected_mask_tensor_shape, mask_tensor_shape))
        if hidden_size % self._num_heads != 0:
            raise ValueError(
                    "The input size (%d) is not a multiple of the number of attention "
                    "heads (%d)" % (hidden_size, self._num_heads))
            

        self._attention_layer = self._attention_layer_type(num_heads=4,key_dim=8, num_rand_blocks=10,from_block_size=10,to_block_size=10,max_rand_mask_length=set_len)
        
        self._attention_dropout = tf.keras.layers.Dropout(
                rate=self._attention_dropout_rate)
        self._attention_layer_norm = tf.keras.layers.LayerNormalization(
                name="self_attention_layer_norm",
                axis=-1,
                epsilon=self._norm_epsilon,
                dtype=tf.float32)
        self._inner_dense = tf.keras.layers.experimental.EinsumDense(
                "abc,cd->abd",
                output_shape=(None, self._inner_size),
                bias_axes="d",
                kernel_initializer=clone_initializer(self._kernel_initializer),
                name="inner")

        self._inner_activation_layer = tf.keras.layers.Activation(
                self._inner_activation)
        self._inner_dropout_layer = tf.keras.layers.Dropout(
                rate=self._inner_dropout)
        self._output_dense = tf.keras.layers.experimental.EinsumDense(
                "abc,cd->abd",
                output_shape=(None, hidden_size),
                bias_axes="d",
                name="output",
                kernel_initializer=clone_initializer(self._kernel_initializer))
        self._output_dropout = tf.keras.layers.Dropout(rate=self._dropout_rate)
        self._output_layer_norm = tf.keras.layers.LayerNormalization(
                name="output_layer_norm",
                axis=-1,
                epsilon=self._norm_epsilon)

        super(TransformerXLBlock, self).build(input_shape)

    def call(self,
             content_stream,
             mask = None,
             relative_position_encoding=None,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        attention_kwargs = dict(
                query=content_stream,
                value=content_stream,
                key=content_stream,
                state = state,
                attention_mask=mask)

        attention_output = self._attention_layer(**attention_kwargs)
        
        attention_stream = attention_output
        input_stream = content_stream
        attention_key = "content_attention"
        attention_output = {}
        
        attention_stream = self._attention_dropout(attention_stream)
        attention_stream = self._attention_layer_norm(attention_stream + input_stream)
        inner_output = self._inner_dense(attention_stream)
        inner_output = self._inner_activation_layer(
                inner_output)
        inner_output = self._inner_dropout_layer(
                inner_output)
        layer_output = self._output_dense(inner_output)
        layer_output = self._output_dropout(layer_output)
        layer_output = self._output_layer_norm(layer_output + attention_stream)
        attention_output[attention_key] = layer_output

        return attention_output

---
# Transformer XL

In [42]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 vocab_size,
                 num_layers,
                 hidden_size,
                 maxlen,
                 embed_dim,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 initializer,
                 tie_attention_biases=True,
                 memory_length=None,
                 reuse_length=None,
                 inner_activation="relu",
                 **kwargs):
        super(TransformerXL, self).__init__(**kwargs)

        self._vocab_size = vocab_size
        self._initializer = initializer
        self._num_layers = num_layers
        self._hidden_size = hidden_size
        self._num_attention_heads = num_attention_heads
        self._head_size = head_size
        self._inner_size = inner_size
        self._inner_activation = inner_activation
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._tie_attention_biases = tie_attention_biases

        self._memory_length = memory_length
        self._reuse_length = reuse_length

        
        self.mask_layer = Create_BigBird_Masks(10)
        
        
        if self._tie_attention_biases:
            attention_bias_shape = [self._num_attention_heads, self._head_size]
        else:
            attention_bias_shape = [self._num_layers, self._num_attention_heads, self._head_size]

        self.transformer_xl_layers = []
        
        for i in range(self._num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(
                            vocab_size=self._vocab_size,
                            hidden_size=self._head_size * self._num_attention_heads,
                            num_attention_heads=self._num_attention_heads,
                            head_size=self._head_size,
                            inner_size=self._inner_size,
                            dropout_rate=self._dropout_rate,
                            attention_dropout_rate=self._attention_dropout_rate,
                            norm_epsilon=1e-12,
                            inner_activation=self._inner_activation,
                            kernel_initializer="variance_scaling",
                            name="layer_%d" % i))

        self.output_dropout = tf.keras.layers.Dropout(rate=self._dropout_rate)


    def call(self,
             content_stream,
             relative_position_encoding,
             mask, 
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        new_mems = []

        if state is None:
            state = [None] * self._num_layers
        for i in range(self._num_layers):
            # cache new mems
            new_mems.append(_cache_memory(content_stream, state[i], self._memory_length, self._reuse_length))

            transformer_xl_layer = self.transformer_xl_layers[i]
            
            mask = self.mask_layer(tf.concat((content_stream, state[i]), axis=1))
            
            transformer_xl_output = transformer_xl_layer(
                    content_stream=content_stream,
                    mask=mask,
                    relative_position_encoding=relative_position_encoding,
                    state=state[i],
                    content_attention_mask=content_attention_mask,
                    query_attention_mask=query_attention_mask,
                    target_mapping=target_mapping)
            content_stream = transformer_xl_output["content_attention"]

        output_stream = content_stream
        return output_stream, new_mems

---
# Xl Model Class

In [43]:
class XlModel(keras.Model):
    def __init__(self, attention_block_size, max_files, encoder, block_size, seq_len_padded, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer):
        super(XlModel, self).__init__()
        
        self.block_size = block_size
        self.seq_len_padded = seq_len_padded
        self.embed_dim = embed_dim
        self.num_heads = num_attention_heads
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.maxlen = maxlen
        self.memory_length = memory_length
        self.max_files = max_files
        self.encoder = encoder
        self.isabs = []
        
        # self.mask_layer = Create_BigBird_Masks(attention_block_size)
            
        self.embedding_layer = Create_Embeddings(encoder)

        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(
                vocab_size=vocab_size,
                num_layers=num_layers,
                hidden_size=hidden_size,
                num_attention_heads=num_attention_heads,
                maxlen=maxlen,
                embed_dim=embed_dim,
                memory_length=memory_length,
                reuse_length=reuse_length,
                head_size=head_size,
                inner_size=inner_size,
                dropout_rate=dropout_rate,
                attention_dropout_rate=attention_dropout_rate,
                initializer=initializer, 
            )
        

        self.pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=1,embed_dim=embed_dim,num_heads=1,use_layernorm=True,pre_layernorm=True, use_keras_mha=True,is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
        
    
    def call(self, x, training=None):        

        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.memory_length, self.embed_dim))
        
        embeddings = self.embedding_layer(x)
            
        linear_transform = self.linear_layer(embeddings)    
        
        mask = None
        
        for i in range(0, self.seq_len_padded, self.block_size):
            
            block = embeddings[:,i:i+self.block_size]
            
            # mask = self.mask_layer(tf.concat((mask, mems), axis=1)
            
            output, mems = self.transformer_xl(content_stream=block, mask=mask, relative_position_encoding=None, state=mems)
                
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        return output

---
# Attention Parameters

In [44]:
attention_block_size = 10

---
# Xl Parameters

In [45]:
#Xl Parameters 
embed_dim = 8
num_layers = 10
hidden_size = 32
num_attention_heads = 4
memory_length = 200
reuse_length = 0
head_size = 8
inner_size = 32
dropout_rate = 0.01
attention_dropout_rate = 0.01
initializer = keras.initializers.RandomNormal(stddev=0.1) 

encoder = pretrained_encoder
epochs = 1000

---
# Create Models

In [46]:
model = XlModel(attention_block_size, max_files, encoder, block_size, seq_len, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Nadam(1e-4), metrics = keras.metrics.SparseCategoricalAccuracy())

In [47]:
(model(train_dataset[0][0])).shape

InvalidArgumentError: Exception encountered when calling layer "big_bird_attention_1" (type BigBirdAttention).

Input to reshape is a tensor with 304000 values, but the requested shape has 464000 [Op:Reshape]

Call arguments received:
  • query=tf.Tensor(shape=(20, 200, 8), dtype=float32)
  • value=tf.Tensor(shape=(20, 200, 8), dtype=float32)
  • key=tf.Tensor(shape=(20, 200, 8), dtype=float32)
  • state=tf.Tensor(shape=(20, 200, 8), dtype=float32)
  • attention_mask=['tf.Tensor(shape=(20, 1, 36, 10, 30), dtype=float32)', 'tf.Tensor(shape=(20, 1, 400, 1), dtype=float32)', 'tf.Tensor(shape=(20, 1, 1, 400), dtype=float32)', 'tf.Tensor(shape=(20, 40, 10), dtype=float32)']
  • kwargs={'training': 'False'}

In [ ]:
history = model.fit(x=train_dataset, validation_data=val_dataset, epochs=epochs, verbose=1)

In [ ]:
model.save_weights("./Saved_Models/Str_Xl_BigBird.h5")

In [ ]:
u = model.evaluate(dataset[3][0], dataset[3][1])

In [ ]:
u = model.predict(dataset[0][0])

In [50]:
u = u.argmax(-1)

In [51]:
u

array([0, 2, 2, 3, 2, 3, 2, 0, 1, 0, 2, 3, 0, 1, 2, 3, 1, 0, 0, 3])

In [52]:
dataset[0][1]

array([0, 2, 4, 4, 3, 4, 2, 0, 1, 0, 2, 4, 0, 1, 3, 4, 1, 0, 0, 4],
      dtype=int32)

In [53]:
random_samples[0:5]

['/data/dna_samples:v1/train/WS-MU-Oct2016_S71_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-HPN-Jul2016_S78_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes21-8-TCR_S22_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-CCE-Sep2015_S60_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley056-NegCtrl_S195_L001_R1_001.db']